Vanilla Neural Network Implementations

-> Pure Python Implemantation using numpy

In [10]:
import numpy as np
import pandas as pd
from datetime import datetime
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve, auc, confusion_matrix

In [11]:
df=pd.read_csv("KaggleV2-May-2016.csv")

In [38]:
# 1. Load and clean the data
def load_data(filename):
    df=pd.read_csv(filename)
    df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
    df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
    df['AppointmentWeekday'] = df['AppointmentDay'].dt.dayofweek
    df['AppointmentGap'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days
    df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
    df['No-show'] = df['No-show'].map({'No': 0, 'Yes': 1})
    df['Neighbourhood'] = LabelEncoder().fit_transform(df['Neighbourhood'])
    return df

# 2. Select features and labels
def get_features_and_labels(df):
    features = ['Age', 'Gender', 'Scholarship', 'Hipertension', 'Diabetes',
                'Alcoholism', 'Handcap', 'SMS_received', 'Neighbourhood',
                'AppointmentWeekday', 'AppointmentGap']
    X = df[features].fillna(0).values
    y = df['No-show'].values.reshape(-1, 1)
    X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    return X, y, features

# 3. Build a simple neural network
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size, lr=0.001):
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
        self.b2 = np.zeros((1, output_size))
        self.lr = lr
    
    def relu(self, x): return np.maximum(0, x)
    def relu_deriv(self, x): return (x > 0).astype(float)
    def sigmoid(self, x): return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    def loss(self, y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.sigmoid(self.Z2)
        return self.A2

    def backward(self, X, y, output):
        m = y.shape[0]
        dZ2 = output - y
        dW2 = np.dot(self.A1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.relu_deriv(self.Z1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2

    def train(self, X_train, y_train, X_val, y_val, epochs):
        start_time = time.time()
        self.epoch_times = []  # Store cumulative time in seconds

        for epoch in range(epochs):
        # Forward pass
            y_pred_train = self.forward(X_train)
            train_loss = self.binary_cross_entropy_loss(y_train, y_pred_train)

        # Backward pass
            self.backward(X_train, y_train, y_pred_train)

        # Validation loss
            y_pred_val = self.forward(X_val)
            val_loss = self.binary_cross_entropy_loss(y_val, y_pred_val)

        # Store losses and time
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.epoch_times.append(time.time() - start_time)

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


    def predict(self, X):
        return (self.forward(X) > 0.5).astype(int)

# 4. Evaluate model
def evaluate(model, X_val, y_val):
    pred = model.predict(X_val)
    proba = model.forward(X_val)
    acc = accuracy_score(y_val, pred)
    f1 = f1_score(y_val, pred)
    cm = confusion_matrix(y_val, pred)
    pr, re, _ = precision_recall_curve(y_val, proba)
    pr_auc = auc(re, pr)
    print(f"Accuracy: {acc:.4f}, F1 Score: {f1:.4f}, PR AUC: {pr_auc:.4f}\nConfusion Matrix:\n{cm}")

# 5. Run everything
df = load_data('KaggleV2-May-2016.csv')
X, y, features = get_features_and_labels(df)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training on {X_train.shape[0]} samples with features: {features}")
model = SimpleNN(input_size=X.shape[1], hidden_size=32, output_size=1, lr=0.001)
start = time.time()
model.train(X_train, y_train, X_val, y_val, epochs=1000)
print(f"Training time: {time.time() - start:.2f} seconds")
evaluate(model, X_val, y_val)
def plot_training_curves_over_time(model):
    plt.figure(figsize=(10, 5))
    plt.plot(model.epoch_times, model.train_losses, label='Training Loss')
    plt.plot(model.epoch_times, model.val_losses, label='Validation Loss')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss over Time')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


Training on 88421 samples with features: ['Age', 'Gender', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'Neighbourhood', 'AppointmentWeekday', 'AppointmentGap']


AttributeError: 'SimpleNN' object has no attribute 'binary_cross_entropy_loss'

-> Pytorch Implementation